# Gaussian Process Modeling

This notebook will run various query selection methods and evaluate performance using a Gaussian Process Model
Methods implemented:
- Mean Acquisition
- Variance Acquisition
- Upper Confidence Bound Acquisition
- Biased Upper Confidence Bound Acquisition
- Random Selection

In [ ]:
#Loading in packages
import numpy as np
import pandas as pd
import GPy
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.utils import shuffle
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import time
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Loading in data and subsetting columns
data = pd.read_csv("TYK2_final.csv")
data = data.drop(['target', 'top_2p'], axis=1)
column_names = ['smiles', 'target', 'active']
data.columns = column_names
data

In [ ]:
#Converting data to fingeraprints and binarizing the "active" column
def smiles_to_fingerprint(smiles, nBits=4096):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=4, nBits=nBits, useChirality=True)
    return list(fp)

data['fingerprint'] = data['smiles'].apply(smiles_to_fingerprint)

data['active'] = data['active'].replace({False: 0, True: 1})

data.to_csv("TYK2_fingerprints.csv")
print(data.head())

# Random Sampling

To perform random sampling, we will randomly select 1,500 of the instances from the dataset to train on, and evaluate on the remaining ligands

In [ ]:
#This function runs random sampling based on the "batch size" (which isn't relevant for random selection)s
#for each seed, we randomly start with 1,000 ligands to train on. While the training set isn't 1,500 ligands yet,
#we get the performance (RMSE, R^2) if the counter is at a multiple of 50,
#and then append new batch size set of data to the training data
#At the end of the seed, we append the list of RMSE and R^2 values to the master list
def random_sampling(batch_size):

    #setting up initial seeds
    seeds = [1, 2, 3]

    #storing master list of RMSE and R^2 values
    rmse_vals = []
    r2_vals = []
  
    #for each seed
    for seed in seeds:

        #storing indices of data points selected (for UMAP analysis)
        final_inds = []

        #storing RMSE and R^2 values every 50 instances added
        rmse_instance =[]
        r2_instance = []

        #getting data (without the smiles column)
        working_data = data.drop(columns="smiles")
        #setting starting size
        size = 1000
        #sampling 'size' amount of data points
        start_data = working_data.sample(n=size, random_state=seed)
        
        #Extracting starting rows (X and labels)
        start_X = np.array([val for val in start_data['fingerprint'].values])
        start_y = start_data['target'].values.reshape(-1,1)
        start_classes = start_data['active'].values.reshape(-1,1)

        #Extracting remaining rows (X and labels)
        remaining_data = working_data.drop(start_data.index)
        remaining_X = np.array([val for val in remaining_data['fingerprint'].values])
        remaining_y = remaining_data['target'].values.reshape(-1,1)
        remaining_classes = remaining_data['active'].values.reshape(-1,1)
        
        #setting counter to 0
        counter = 0

        #for as long as our training set is 1,500 ligands
        while len(start_X) <= 1500:

            #record RMSE and R^2 every 50 instances added
            if counter % 50 == 0:
                #creating kernel
                k = GPy.kern.Linear(len(start_X[0]))

                #training and optimizing GP regression model
                m = GPy.models.GPRegression(start_X, start_y, k)
               
                #Predicting on final 20%
                pred_means, pred_vars = m.predict(remaining_X)

                #Getting rmse and r^2 score
                rmse_instance.append(np.sqrt(mean_squared_error(remaining_y, pred_means)))
                r2_instance.append(r2_score(remaining_y, pred_means))
            
            #once our training size is 1,500, we exit the loop
            if len(start_X) >= 1500:
                break
        
            #sampling batch size random row(s) and saving indices for UMAP
            rand_rows = remaining_data.sample(n=batch_size, random_state=seed)
            final_inds.append(rand_rows.index.to_list())

            #adding the row of the selected index to the starting data
            start_data = pd.concat([start_data, rand_rows])
            start_X = np.array([val for val in start_data['fingerprint'].values])
            start_y = start_data['target'].values.reshape(-1,1)


            #removing the row of the selected index from the remaining data
            remaining_data = working_data.drop(start_data.index)
            remaining_X = np.array([val for val in remaining_data['fingerprint'].values])
            remaining_y = remaining_data['target'].values.reshape(-1,1)

            #incrementing counter by batch size
            counter += batch_size
        
        #append the current seed's list of values to the master list
        rmse_vals.append(rmse_instance)
        r2_vals.append(r2_instance)
        
    #return master lists and list of indices (just from seed 3)
    return rmse_vals, r2_vals, final_inds

In [ ]:
#This cell runs the random sampling simulations for varying batch sizes (1, 25, and 50), and saves the RMSE and R^2 values, as well as the runtimes
#The results are saved into a dictionary for each batch
batch_sizes = [1, 25, 50]
rand_rmse_dict = {}
rand_r2_dict = {}

#final dataframe that saves both dictionaries
final_rand_data = pd.DataFrame()

#for each batch, run the simulations and save the results
for batch in batch_sizes:

    #getting runtime
    start = time.time()
    #run simulation
    rand_rmse, rand_r2, final_inds = random_sampling(batch)
    #get ending runtime
    end = time.time()

    #Saving runtime (divided by 3 to get per-seed results)
    final_rand_data[f"{batch}_runtime"] = (end - start) / 3
    
    #calculating mean and std dev of RMSE across the 3 seeds
    rand_rmse_mean = np.mean(rand_rmse, axis=0)
    rand_rmse_stdev = np.std(rand_rmse, axis=0)
    print("mean:", rand_rmse_mean)
    print("standard deviation:", rand_rmse_stdev)
    #saving data to the dictionary
    rand_rmse_dict[batch] = (rand_rmse, rand_rmse_mean, rand_rmse_stdev)
    #adding it to a new column in the dataframe for the specific batch
    final_rand_data[f"{batch}_rmse_mean"] = rand_rmse_mean
    final_rand_data[f"{batch}_rmse_stdev"] = rand_rmse_stdev

    #calculating mean and std dev of R^2 across the 3 seeds
    rand_r2_mean = np.mean(rand_r2, axis=0)
    rand_r2_stdev = np.std(rand_r2, axis=0)
    print("mean:", rand_r2_mean)
    print("standard deviation:", rand_r2_stdev)
    #saving data to dictionary
    rand_r2_dict[batch] = (rand_r2, rand_r2_mean, rand_r2_stdev)
    #adding it to a new column in the dataframe for the specific abtch
    final_rand_data[f"{batch}_r2_mean"] = rand_r2_mean
    final_rand_data[f"{batch}_r2_stdev"] = rand_r2_stdev

    #saving indices into a csv file for the specific batch
    final_inds = pd.Series(final_inds)
    final_inds.to_csv(f"{batch}_final_inds_random.csv")

#writing final dataframe to a csv
final_rand_data.to_csv("final_data_random.csv")

# Uncertainity Sampling
## - Mean, Variance, UCB, Biased UCB

The remaining cells run the above 4 uncertainty sampling methods, starting with 1,000 ligands and stopping once 1,500 ligands have been added
to the training dataset

In [ ]:
#This function runs a random forest classifier on the training datafor the BIASED UCB method. 
#It returns the probability of being active/inactive for all of the unseen datapoints

def active_prediction(start_X, classes, remaining_X, seed):

    #creating random forest classifier
    m = RandomForestClassifier(n_estimators=20, random_state=seed)
    #fitting to training data
    m.fit(start_X, classes.ravel())
    #predicting on unseen data
    probs = m.predict_proba(remaining_X)
    #returning probabilities
    return probs

In [ ]:
#This function runs sequential model-based optimization. This function works by training a GP model on the start data, 
#and return the GP outputs (mean vector and variance vector) of all unseen data
def smbo(start_X, start_y, remaining_X):
    
    #creating a linear GP kernel based on the number of columns in the dataframe (in this case 1)
    k = GPy.kern.Linear(len(start_X[0]))
    #fitting GP regression model to the start data 
    m = GPy.models.GPRegression(start_X, start_y, k)
    #predict GP mean and variance of the unseen data
    mean, var = m.predict(remaining_X, full_cov=False)
    #return the GP mean and variance of the unseen data
    return mean, var

In [ ]:
#This function runs uncertainty sampling based on the batch size (using the batch_size parameter)
#for each seed, we randomly start with 1,000 ligands to train on. While the training set isn't 1,500 ligands yet,
#we get the performance (RMSE, R^2) if the counter is at a multiple of 50,
#and then append new batch size set of data to the training data based on one of 4 query seleciton strategies (using the flag parameter):
#   - Mean, Variance, UCB, Biased UCB
#At the end of the seed, we append the list of RMSE and R^2 values to the master list
def uncertainty_sampling(flag, batch_size):
    #testing for 3 seeds
    seeds = [1,2,3]

    #master list of RMSE and R^2 values
    rmse_vals = []
    r2_vals = []

    #for each seed...
    for seed in seeds:

        #list of selected indices by query selection method
        final_inds = []

        #current seed's list of RMSE and R^2 values
        rmse_instance =[]
        r2_instance = []
      
        #dropping unecessary columns
        working_data = data.drop(columns="smiles")
        #setting starting size to 1,000
        size = 1000
        #randomly selecting the starting 1,000 ligands
        start_data = working_data.sample(n=size, random_state=seed)
        
        #subsetting the starting data: X, ligands, and classes (used for biased UCB method)
        start_X = np.array([val for val in start_data['fingerprint'].values])
        start_y = start_data['target'].values.reshape(-1,1)
        start_classes = start_data['active'].values.reshape(-1,1)

        #subsetting the remaining unseen data: X and ligands
        remaining_data = working_data.drop(start_data.index)
        remaining_X = np.array([val for val in remaining_data['fingerprint'].values])
        remaining_y = remaining_data['target'].values.reshape(-1,1)

        #set initial variables for calculating UCB
        Dsize = len(start_X) #size of the dataset
        bo_lambda = 0.1 #how much importance to place on variance in UCB
        bo_iters = 1 #number of iterations

        #calculate beta constant from the course-provided material
        beta = 2 * math.log(Dsize * math.pow(bo_iters,2) * math.pow(np.pi,2) / (6 * bo_lambda) )

        #getting initial counter
        counter = 0

        #Until we sample another 500 ligands...
        while len(start_X) <= 1500:
            
            #calculate RMSE adn R^2 evry 50 ligands
            if counter % 50 == 0:
                #creating linear kernel
                k = GPy.kern.Linear(len(start_X[0]))

                #training and optimizing GP regression model
                m = GPy.models.GPRegression(start_X, start_y, k)

                #Predicting on unseen data
                pred_means, pred_vars = m.predict(remaining_X)

                #Getting rmse and R^2 score
                rmse_instance.append(np.sqrt(mean_squared_error(remaining_y, pred_means)))
                r2_instance.append(r2_score(remaining_y, pred_means))
            
            #once we get to 1,500 ligands, exit the current seed
            if len(start_X) >= 1500:
                break

            #run sequential model-based optimization and get the GP parameters to select the next instance
            mean, var = smbo(start_X, start_y, remaining_X)

            #depending on the selection function, we calculate a specific alpha_full value
            if flag == "ucb":
                #get the UCB value at each x: (mean + beta*var)
                alpha_full = mean + math.sqrt(beta) * var
                alpha_full = [item for sublist in alpha_full for item in sublist]

            elif flag == "biased":
                #get the biased UCB value at each x: (mean + beta*var) * prob_active

                #get the probability of each unseen data being active
                probs = active_prediction(start_X, start_classes, remaining_X, seed)
                probs = [prob[1] for prob in probs]

                #calculate base ucb
                alpha_full = mean + math.sqrt(beta) * var
                alpha_full = [item for sublist in alpha_full for item in sublist]

                #multiply by prob_active
                alpha_full = np.array(alpha_full) * probs
                alpha_full = alpha_full.tolist()

            elif flag == "mean":
                #just save the predicted mean value
                alpha_full = mean
                alpha_full = [item for sublist in alpha_full for item in sublist]
            else:
                # just save the predicted variance calue
                alpha_full = [item for sublist in var for item in sublist]
    
            #get the index for the row with the largest UCB/biased UCB/mean/var by sorting the list and maintaining the correct index
            sorted_rows = sorted(range(len(alpha_full)), key=lambda x: alpha_full[x], reverse=True)
            #getting indices of selected data
            inds = sorted_rows[:batch_size]

            #extracting the new data rows to add
            rows_to_add = remaining_data.iloc[inds]
            #saving the indices to the list for UMAP analysis
            for idx in rows_to_add.index:
                final_inds.append(idx)

            #adding new row(s) to the starting data and re-separating X, ligands, and classes
            start_data = pd.concat([start_data, rows_to_add])
            start_X = np.array([val for val in start_data['fingerprint'].values])
            start_y = start_data['target'].values.reshape(-1,1)
            start_classes = start_data['active'].values.reshape(-1,1)

            #removing row(s) from the remaining data and re-separating X, ligands
            remaining_data = working_data.drop(start_data.index)
            remaining_X = np.array([val for val in remaining_data['fingerprint'].values])
            remaining_y = remaining_data['target'].values.reshape(-1,1)

            #increment counter
            counter += batch_size

        #appending the current seed's values to the master list
        rmse_vals.append(rmse_instance)
        r2_vals.append(r2_instance)
        
    #returning the master lists and the indices (just from seed 3)
    return rmse_vals, r2_vals, final_inds

In [ ]:
#This cell runs the UCB simulations for varying batch sizes (1, 25, and 50), and saves the RMSE and R^2 values, as well as the runtimes
#The results are saved into a dictionary for each batch and then stored into a dataframe for exporting

#list of batch sizes
batch_sizes = [1, 25, 50]
flag = "ucb"

#dictionaries for RMSE and R^2 values
ucb_rmse_dict = {}
ucb_r2_dict = {}

#final dataframe storing all scores and runtimes
final_ucb_data = pd.DataFrame()

#for each batch
for batch in batch_sizes:

    #getting start time
    start = time.time()
    #running UCB
    ucb_rmse, ucb_r2, final_inds = uncertainty_sampling(flag, batch)
    #getting ending time
    end = time.time()

    #append runtime to dataframe
    final_ucb_data[f"{batch}_runtime"] = end - start

    #calculate the mean and std. dev of RMSE across 3 seeds
    ucb_rmse_mean = np.mean(ucb_rmse, axis=0)
    ucb_rmse_stdev = np.std(ucb_rmse, axis=0)
    print("mean:", ucb_rmse_mean)
    print("standard deviation:", ucb_rmse_stdev)
    #add results to dictionary
    ucb_rmse_dict[batch] = (ucb_rmse, ucb_rmse_mean, ucb_rmse_stdev)
    
    #add dictionary to new columns in dataframe
    final_ucb_data[f"{batch}_rmse_mean"] = ucb_rmse_mean
    final_ucb_data[f"{batch}_rmse_stdev"] = ucb_rmse_stdev

    #calculate the mean and std. dev of R^2 across 3 seeds
    ucb_r2_mean = np.mean(ucb_r2, axis=0)
    ucb_r2_stdev = np.std(ucb_r2, axis=0)
    print("mean:", ucb_r2_mean)
    print("standard deviation:", ucb_r2_stdev)
    #add results to dictionary
    ucb_r2_dict[batch] = (ucb_r2, ucb_r2_mean, ucb_r2_stdev)
    #add dictionaries to new columns in dataframe
    final_ucb_data[f"{batch}_r2_mean"] = ucb_r2_mean
    final_ucb_data[f"{batch}_r2_stdev"] = ucb_r2_stdev

    #saving selected indices to a csv
    final_inds = pd.Series(final_inds)
    final_inds.to_csv(f"{batch}_final_inds_ucb.csv")

#writing final dataframe
final_ucb_data.to_csv("final_data_ucb.csv")


In [ ]:
#This cell runs the biased UCB simulations for varying batch sizes (1, 25, and 50), and saves the RMSE and R^2 values, as well as the runtimes
#The results are saved into a dictionary for each batch and then stored into a dataframe for exporting

#list of batch sizes
batch_sizes = [1, 25, 50]
flag = "biased"

#dictionaries storing RMSE and R^2 values
ucb_rmse_dict = {}
ucb_r2_dict = {}

#final dataframe to store results
final_ucb_biased_data = pd.DataFrame()

#for each batch size
for batch in batch_sizes:

    #getting start time
    start = time.time()
    #running biased UCB seleciton
    ucb_rmse, ucb_r2, final_inds = uncertainty_sampling(flag, batch)
    #getting end time
    end = time.time()

    #appending runtime to dataframe
    final_ucb_biased_data[f"{batch}_runtime"] = end - start

    #calculating mean and std. dev of RMSE across seeds
    ucb_rmse_mean = np.mean(ucb_rmse, axis=0)
    ucb_rmse_stdev = np.std(ucb_rmse, axis=0)
    print("mean:", ucb_rmse_mean)
    print("standard deviation:", ucb_rmse_stdev)
    #adding results to dictionary
    ucb_rmse_dict[batch] = (ucb_rmse, ucb_rmse_mean, ucb_rmse_stdev)
    #saving data to final dataframe
    final_ucb_biased_data[f"{batch}_rmse_mean"] = ucb_rmse_mean
    final_ucb_biased_data[f"{batch}_rmse_stdev"] = ucb_rmse_stdev

    #calculating mean and std. dev of R^2 across seeds
    ucb_r2_mean = np.mean(ucb_r2, axis=0)
    ucb_r2_stdev = np.std(ucb_r2, axis=0)
    print("mean:", ucb_r2_mean)
    print("standard deviation:", ucb_r2_stdev)
    #adding results to dictionary
    ucb_r2_dict[batch] = (ucb_r2, ucb_r2_mean, ucb_r2_stdev)
    #savind data to final dataframe
    final_ucb_biased_data[f"{batch}_r2_mean"] = ucb_r2_mean
    final_ucb_biased_data[f"{batch}_r2_stdev"] = ucb_r2_stdev

    #save selected indices
    final_inds = pd.Series(final_inds)
    final_inds.to_csv(f"{batch}_final_inds_ucb_biased.csv")

#save final dataframe
final_ucb_biased_data.to_csv("final_data_ucb_biased.csv")


In [ ]:
#This cell runs the mean simulations for varying batch sizes (1, 25, and 50), and saves the RMSE and R^2 values, as well as the runtimes
#The results are saved into a dictionary for each batch and then stored into a dataframe for exporting

#batch sizes
batch_sizes = [1, 25, 50]
flag = "mean"
#dictionary to store RMSE adn R^2 values
mean_rmse_dict = {}
mean_r2_dict = {}
#final dataframe with results
final_mean_data = pd.DataFrame()

for batch in batch_sizes:
    #get start time
    start = time.time()
    #run mean simulation
    mean_rmse, mean_r2, final_inds = uncertainty_sampling(flag, batch)
    #get finish time
    end = time.time()
    #save runtime to dataframe
    final_mean_data[f"{batch}_runtime"] = end - start
    
    #calculating mean and std. dev of RMSE across seeds
    mean_rmse_mean = np.mean(mean_rmse, axis=0)
    mean_rmse_stdev = np.std(mean_rmse, axis=0)
    print("mean:", mean_rmse_mean)
    print("standard deviation:", mean_rmse_stdev)
    #saving results to dictionary
    mean_rmse_dict[batch] = (mean_rmse, mean_rmse_mean, mean_rmse_stdev)
    #adding dictionary as new columns to dataframe
    final_mean_data[f"{batch}_rmse_mean"] = mean_rmse_mean
    final_mean_data[f"{batch}_rmse_stdev"] = mean_rmse_stdev

    #calculating mean and std. dev of R^2 across seeds
    mean_r2_mean = np.mean(mean_r2, axis=0)
    mean_r2_stdev = np.std(mean_r2, axis=0)
    print("mean:", mean_r2_mean)
    print("standard deviation:", mean_r2_stdev)
    #saving results to dictionary
    mean_r2_dict[batch] = (mean_r2, mean_r2_mean, mean_r2_stdev)
    #adding dictionary as new columns to dataframe
    final_mean_data[f"{batch}_r2_mean"] = mean_r2_mean
    final_mean_data[f"{batch}_r2_stdev"] = mean_r2_stdev

    #saving final selected indices
    final_inds = pd.Series(final_inds)
    final_inds.to_csv(f"{batch}_final_inds_mean.csv")

#save final dataframe
final_mean_data.to_csv("final_data_mean.csv")

In [ ]:
#This cell runs the variance simulations for varying batch sizes (1, 25, and 50), and saves the RMSE and R^2 values, as well as the runtimes
#The results are saved into a dictionary for each batch and then stored in a dataframe for exporting

#batch sizes
batch_sizes = [1, 25, 50]
flag = "var"
#dictionaries storing RMSE and R^2 values
var_rmse_dict = {}
var_r2_dict = {}

#final dataframe storing value and runtimes
final_var_data = pd.DataFrame()

#for each batch...
for batch in batch_sizes:
    
    #get start time
    start = time.time()
    #run variance simulations
    var_rmse, var_r2, final_inds = uncertainty_sampling(flag, batch)
    #get end time
    end = time.time()
    #adding run time to dataframe
    final_var_data[f"{batch}_runtime"] = end - start

    #getting mean and std. dev of RMSE across seeds
    var_rmse_mean = np.mean(var_rmse, axis=0)
    var_rmse_stdev = np.std(var_rmse, axis=0)
    print("mean:", var_rmse_mean)
    print("standard deviation:", var_rmse_stdev)
    #adding values to dictionary 
    var_rmse_dict[batch] = (var_rmse, var_rmse_mean, var_rmse_stdev)
    #adding dictionary to dataframe as new columns
    final_var_data[f"{batch}_rmse_mean"] = var_rmse_mean
    final_var_data[f"{batch}_rmse_stdev"] = var_rmse_stdev

    #getting mean and std. dev of R^2 across seeds
    var_r2_mean = np.mean(var_r2, axis=0)
    var_r2_stdev = np.std(var_r2, axis=0)
    print("mean:", var_r2_mean)
    print("standard deviation:", var_r2_stdev)
    #adding values to dictionary
    var_r2_dict[batch] = (var_r2, var_r2_mean, var_r2_stdev)
    #adding dictionary to dataframe as new columns
    final_var_data[f"{batch}_r2_mean"] = var_r2_mean
    final_var_data[f"{batch}_r2_stdev"] = var_r2_stdev

    #saving final selected indices per batch
    final_inds = pd.Series(final_inds)
    final_inds.to_csv(f"{batch}_final_inds_var.csv")

#saving final dataframe of results
final_var_data.to_csv("final_data_var.csv")